In [2]:
import pandas as pd
import os
from tqdm import tqdm

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="gfdg")

In [3]:
rootdir = r'C:\\Sohum\\Home\\CodeHome\\Python\\optimization_course\\energy_sytems_optimization\\preprocessing\\bulk_downloader\\data\\'
def get_solar_wind_at_year(yr):

    '''saves solar and wind data for certain years into 1 csv file'''
        
    lst1=[]
    for subdir, dirs, files in os.walk(rootdir+"wind"):

        if (yr+".csv" in files):
            lst1.append(subdir.split("\\")[-2]+"_"+subdir.split("\\")[-1])


    lst2=[]
    for subdir, dirs, files in os.walk(rootdir+"solar"):

        if (yr+".csv" in files):
            lst2.append(subdir.split("\\")[-2]+"_"+subdir.split("\\")[-1])
        


    lst=list(set(lst1).intersection(lst2))
    frames=[]

    for place in lst:

        file_path=rootdir+"solar/"+place.split("_")[0]+"/"+place.split("_")[1] +"/"+yr+".csv"
        df=pd.read_csv(file_path)
        col_name=place
        if (len(frames)!=0):
            try:

                del df['ob_time']
            except:
                del df['ob_end_time']
            df.columns = [col_name+".Solar"]
        else:
            df.columns = ['ob_time',col_name+".Solar"]
        frames.append(df)
        res=pd.concat(frames,axis=1)
        res.fillna(0,inplace=True)
        file_path=rootdir+"wind/"+place.split("_")[0]+"/"+place.split("_")[1] +"/"+yr+".csv"
        df=pd.read_csv(file_path)
        col_name=place
        if (len(frames)!=0):
            try:

                del df['ob_time']
            except:
                del df['ob_end_time']
            df.columns = [col_name+".Wind"]
        else:
            df.columns = ['ob_time',col_name+".Wind"]
        frames.append(df)
        res=pd.concat(frames,axis=1)
        res.fillna(0,inplace=True)
        res.to_csv("output/"+yr+".csv",index=False)


In [4]:

# for i in range(2015,2020):
#     get_solar_wind_at_year(str(i))
    


In [5]:
df= pd.read_csv("output/"+"2015"+".csv")



In [6]:
place_lst=list(set(list(el.replace(".Solar","").replace(".Wind","").replace("-no-2","") for el in df.columns)))

In [7]:
def get_latitude(place):
    el=geolocator.geocode(place)
    if el is not None:
        if 50<el.latitude<60:
            return el.latitude

    else:
        print("CANT FIND "+place)
def get_longitude(place):
    el=geolocator.geocode(place)
    if el is not None:
        if -8<el.longitude<3:
    
            return el.longitude  

        else:
            print("CANT FIND "+place)

In [8]:
print(get_latitude("uk"))

54.7023545


In [9]:
lat_long_lst=[]
for i in (range(len(place_lst))):

    lat1=get_latitude(place_lst[i].split("_")[1].replace("-"," ")+" uk")
    long1=get_longitude(place_lst[i].split("_")[1].replace("-"," ")+" uk")

    if lat1 is not None:
        lat_long_lst.append([lat1,long1])

    else:

        lat2=get_latitude(place_lst[i].split("_")[1].split("-")[0]+" uk")
        long2=get_longitude(place_lst[i].split("_")[1].split("-")[0]+" uk")

        if lat2 is not None:
            lat_long_lst.append([lat2,long2])

        else:
            print("DANGER ",place_lst[i])
            if (place_lst[i]=="shetland_lerwick"):
                lat2=60.5297
                long2=1.2659
                lat_long_lst.append([lat2,long2])



IndexError: list index out of range

In [ ]:
len(place_lst)==len(lat_long_lst)
dictionary = dict(zip(place_lst, lat_long_lst))
del dictionary["ob_time"]
df=pd.DataFrame(dictionary)
df=df.T


In [ ]:
df.columns=["Latitude","Longitude"]
df.to_csv("supply_lat_long.csv")

In [ ]:
df= pd.read_csv("../../raw_data/demand/Subnational_electricity_consumption_statistics_2017.csv")

In [ ]:
place_lst = df['Local Authority'].tolist()
lat_long_lst=[]
for i in (range(len(place_lst))):
    
    lat1=get_latitude(place_lst[i]+" uk")
    long1=get_longitude(place_lst[i]+" uk")

    if lat1 is not None:
        lat_long_lst.append([lat1,long1])

    else:
        print("DANGER ",place_lst[i])


DANGER  Isles of Scilly
DANGER  Shetland Islands
CANT FIND Unallocated uk
DANGER  Unallocated


In [ ]:
len(place_lst)==len(lat_long_lst)
dictionary = dict(zip(place_lst, lat_long_lst))
df=pd.DataFrame(dictionary)
df=df.T

In [ ]:
df.columns=["Latitude","Longitude"]

df.to_csv("demand_lat_long.csv")

In [12]:
import folium

my_map = folium.Map(location=[51.5074, 0.1278],
                    zoom_start = 5) # for UK 

df_supply=pd.read_csv("supply_lat_long.csv")
df_supply.fillna(0,inplace=True)
for idx,row in df_supply.iterrows():
    # folium.Marker( location=[ (row["Latitude"]), (row["Longitude"]) ], fill_color='red', radius=1 ).add_to( my_map )
    folium.CircleMarker(location=[ (row["Latitude"]), (row["Longitude"]) ],
                        radius=1,
                        weight=5).add_to(my_map)

df_demand=pd.read_csv("demand_lat_long.csv")
df_demand.fillna(0,inplace=True)
# for idx,row in df_demand.iterrows():
#     # folium.Marker( location=[ (row["Latitude"]), (row["Longitude"]) ], fill_color='red', radius=1 ).add_to( my_map )
#     folium.CircleMarker(location=[ (row["Latitude"]), (row["Longitude"]) ],
#                         radius=1,
#                         color='red',
#                         weight=5).add_to(my_map)
my_map

In [22]:
def dist_between_two_lat_lon(*args):
    from math import asin, cos, radians, sin, sqrt
    lat1, lat2, long1, long2 = map(radians, args)

    dist_lats = abs(lat2 - lat1) 
    dist_longs = abs(long2 - long1) 
    a = sin(dist_lats/2)**2 + cos(lat1) * cos(lat2) * sin(dist_longs/2)**2
    c = asin(sqrt(a)) * 2
    radius_earth = 6378 # the "Earth radius" R varies from 6356.752 km at the poles to 6378.137 km at the equator.
    return c * radius_earth

def find_closest_lat_lon(data, v):
    try:
        return min(data, key=lambda p: dist_between_two_lat_lon(v['Latitude'],p['Latitude'],v['Longitude'],p['Longitude']))
    except TypeError:
        print('Not a list or not a number.')
    
# city = {'lat_key': value, 'lon_key': value}  # type:dict()
new_york = {'lat': 40.712776, 'lon': -74.005974}
washington = {'lat': 47.751076,  'lon': -120.740135}
san_francisco = {'lat': 37.774929, 'lon': -122.419418}

city_list = [new_york, washington, san_francisco]
supply_dict=df_supply.to_dict("records")
demand_dict=df_demand.to_dict("records")

city_to_find = {'Latitude': 29.760427, 'Longitude': -95.369804}  # Houston


subset_of_demand=[]
for i in range(len(supply_dict)):
    
    subset_of_demand.append([supply_dict[i],find_closest_lat_lon(demand_dict, supply_dict[i])])
subset_of_demand


[[{'Unnamed: 0': 'sutherland_altnaharra',
   'Latitude': 58.2822128,
   'Longitude': -4.4412206},
  {'Unnamed: 0': 'Falkirk', 'Latitude': 57.5066357, 'Longitude': -5.0038367}],
 [{'Unnamed: 0': 'cambridgeshire_monks-wood',
   'Latitude': 51.75449755,
   'Longitude': -1.182550777346949},
  {'Unnamed: 0': 'Oxford', 'Latitude': 51.7520131, 'Longitude': -1.2578499}],
 [{'Unnamed: 0': 'greater-london_kenley-airfield',
   'Latitude': 51.3227524,
   'Longitude': -0.1037854},
  {'Unnamed: 0': 'Croydon', 'Latitude': 51.3713049, 'Longitude': -0.101957}],
 [{'Unnamed: 0': 'gwynedd_valley',
   'Latitude': 51.48645995,
   'Longitude': 0.0364625462746595},
  {'Unnamed: 0': 'East',
   'Latitude': 51.49455235,
   'Longitude': -0.0173025374143791}],
 [{'Unnamed: 0': 'cumbria_shap',
   'Latitude': 54.51368495,
   'Longitude': -2.673918098343876},
  {'Unnamed: 0': 'Eden',
   'Latitude': 54.6056481,
   'Longitude': -2.671522203148835}],
 [{'Unnamed: 0': 'bedfordshire_bedford',
   'Latitude': 52.1363806,
 

In [23]:

for i in range(len(subset_of_demand)):
    # folium.Marker( location=[ (row["Latitude"]), (row["Longitude"]) ], fill_color='red', radius=1 ).add_to( my_map )
    folium.CircleMarker(location=[ (subset_of_demand[int(i)]["Latitude"]), (subset_of_demand[int(i)]["Longitude"]) ],
                        radius=1,
                        color="red",
                        weight=5).add_to(my_map)


supply_df=pd.DataFrame(supply_dict)
demand_df=pd.DataFrame(subset_of_demand)
supply_df

TypeError: list indices must be integers or slices, not str

In [21]:
demand_df

,Unnamed: 0,Latitude,Longitude
0,Falkirk,57.506636,-5.003837
1,Oxford,51.752013,-1.257850
2,Croydon,51.371305,-0.101957
3,East,51.494552,-0.017303
4,Eden,54.605648,-2.671522
...,...,...,...
73,South Lanarkshire,57.148243,-2.092809
74,Scottish Borders,55.299818,-4.645988
75,South Somerset,50.984305,-2.776156
76,Denbighshire,53.216510,-3.140219
